In [1]:
import psycopg2 
import pandas as pd

In [2]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
except psycopg2.Error as e:
    print("Error: Could not able to establish connection with Postgres database")
    print(e)

In [3]:
try:
    cur = conn.cursor()    
except psycopg2.Error as e:
    print("Error: Could not get cursor to the Database")
    print(e)

In [4]:
conn.set_session(autocommit=True)
try:
    cur.execute("create database airbnb")   
except psycopg2.Error as e:
    print(e)

database "airbnb" already exists



In [5]:
def create_database():
    # connect to deafult database
    # conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=root")
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE airbnb")
    cur.execute("CREATE DATABASE airbnb")

    # close connection to default database
    conn.close()

    # connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=airbnb user=postgres password=root")
    cur = conn.cursor()

    return cur, conn

In [6]:
def drop_table(cur,conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [7]:
def create_table(cur,conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [8]:
airnb = pd.read_csv("dataset_proj01/airnb.csv")

In [9]:
airnb.head()

,Title,Detail,Date,Price(in dollar),Offer price(in dollar),Review and rating,Number of bed
0,"Chalet in Skykomish, Washington, US",Sky Haus - A-Frame Cabin,Jun 11 - 16,306.00,229.00,4.85 (531),4 beds
1,"Cabin in Hancock, New York, US",The Catskill A-Frame - Mid-Century Modern Cabin,Jun 6 - 11,485.00,170.00,4.77 (146),4 beds
2,"Cabin in West Farmington, Ohio, US",The Triangle: A-Frame Cabin for your city retreat,Jul 9 - 14,119.00,522.00,4.91 (515),4 beds
3,"Home in Blue Ridge, Georgia, US",*Summer Sizzle* 5 Min to Blue Ridge* Pets* Hot...,Jun 11 - 16,192.00,348.00,4.94 (88),5 beds
4,"Treehouse in Grandview, Texas, US",Luxury Treehouse Couples Getaway w/ Peaceful V...,Jun 4 - 9,232.00,196.00,4.99 (222),1 queen bed


In [10]:
airnbDesert = pd.read_csv("dataset_proj01/airnb_desert.csv", encoding="latin1")

In [11]:
airnbDesert.head()

,Desert name,Date,Price(In dollar),Details,Rating
0,"Mhamid, Morocco",May 1  29,479.00,Near Sahara Desert,4.79
1,"Aqaba City, Jordan",May 1  29,"2,168.00",Near Arabian Desert,4.92
2,"Tamesluht, Morocco",May 1  29,"17,752.00","9,404 kilometers away",4.95
3,"Al Bairat, Egypt",May 1  29,"1,982.00",Near Sahara Desert,4.88
4,"Tamesluht, Morocco",May 1  29,"17,752.00","9,404 kilometers away",4.87


In [12]:
airnbLuxe = pd.read_csv("dataset_proj01/airnb_luxe.csv", encoding="latin1")

In [13]:
airnbLuxe.head()

,Luxe name,Date,Price(In dollar),Distance
0,"Koh Samui, Thailand",May 1  29,"89,600.00","1,880 kilometers away"
1,"Koh Samui, Thailand",May 1  29,"78,459.00","1,880 kilometers away"
2,"Koh Samui, Thailand",May 1  29,"53,200.00","1,881 kilometers away"
3,"Koh Samui, Thailand",May 1  29,"35,000.00","1,880 kilometers away"
4,"Nathon, Thailand",May 1  29,"19,656.00","1,872 kilometers away"


In [14]:
airnb.head()

,Title,Detail,Date,Price(in dollar),Offer price(in dollar),Review and rating,Number of bed
0,"Chalet in Skykomish, Washington, US",Sky Haus - A-Frame Cabin,Jun 11 - 16,306.00,229.00,4.85 (531),4 beds
1,"Cabin in Hancock, New York, US",The Catskill A-Frame - Mid-Century Modern Cabin,Jun 6 - 11,485.00,170.00,4.77 (146),4 beds
2,"Cabin in West Farmington, Ohio, US",The Triangle: A-Frame Cabin for your city retreat,Jul 9 - 14,119.00,522.00,4.91 (515),4 beds
3,"Home in Blue Ridge, Georgia, US",*Summer Sizzle* 5 Min to Blue Ridge* Pets* Hot...,Jun 11 - 16,192.00,348.00,4.94 (88),5 beds
4,"Treehouse in Grandview, Texas, US",Luxury Treehouse Couples Getaway w/ Peaceful V...,Jun 4 - 9,232.00,196.00,4.99 (222),1 queen bed


In [15]:
cur, conn = create_database()

In [16]:
airnb.columns

Index(['Title', 'Detail', 'Date', 'Price(in dollar)', 'Offer price(in dollar)',
       'Review and rating', 'Number of bed'],
      dtype='object')

In [17]:
air_table_create = ("""CREATE TABLE IF NOT EXISTS nbair(
    Title VARCHAR,
    Detail VARCHAR,
    Date VARCHAR,
    Price FLOAT,
    Offer_price FLOAT,
    Review_and_Rating FLOAT,
    Number_of_bed VARCHAR
)""")

In [18]:
conn.rollback()

In [19]:
cur.execute(air_table_create)
conn.commit()

In [20]:
nbair_table_insert = ("""INSERT INTO nbair(
Title,
Detail,
Date,
Price,
Offer_price,
Review_and_Rating,
Number_of_bed)
VALUES (%s, %s, %s, %s, %s, %s, %s)
""")

In [114]:
for i, row in airnb.iterrows():
    rating = row['Review and rating']
    rating = rating.split()[0]  
    rating = float(rating)
    row['rating'] = rating

    price = row['Price(in dollar)']
    price = price.split()[0]  
    price = float(price)
    row['rating'] = price

    offer_price = row['Offer price(in dollar)']
    offer_price = offer_price.split()[0]  
    offer_price = float(offer_price)
    row['rating'] = offer_price


    cur.execute(nbair_table_insert, list(row))

TypeError: not all arguments converted during string formatting

In [93]:
airnb.columns

Index(['Title', 'Detail', 'Date', 'Price(in dollar)', 'Offer price(in dollar)',
       'Review and rating', 'Number of bed'],
      dtype='object')

In [21]:
print(airnb.head())

                                 Title   
0  Chalet in Skykomish, Washington, US  \
1       Cabin in Hancock, New York, US   
2   Cabin in West Farmington, Ohio, US   
3      Home in Blue Ridge, Georgia, US   
4    Treehouse in Grandview, Texas, US   

                                              Detail         Date   
0                           Sky Haus - A-Frame Cabin  Jun 11 - 16  \
1    The Catskill A-Frame - Mid-Century Modern Cabin   Jun 6 - 11   
2  The Triangle: A-Frame Cabin for your city retreat   Jul 9 - 14   
3  *Summer Sizzle* 5 Min to Blue Ridge* Pets* Hot...  Jun 11 - 16   
4  Luxury Treehouse Couples Getaway w/ Peaceful V...    Jun 4 - 9   

  Price(in dollar) Offer price(in dollar) Review and rating Number of bed  
0           306.00                 229.00        4.85 (531)        4 beds  
1           485.00                 170.00        4.77 (146)        4 beds  
2           119.00                 522.00        4.91 (515)        4 beds  
3           192.00          